In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data=pd.read_csv('../input/planets-dataset/planet/planet/train_classes.csv')
data.head()

In [ ]:
labels=set()

In [ ]:
def split_tags(t):
    [labels.add(tag) for tag in t.split()]

In [ ]:
data2 = data.copy()

In [ ]:
data2['tags'].apply(split_tags)
labels=list(labels)
labels

In [ ]:
for tag in labels:
    data2[tag]=data2['tags'].apply(lambda x:1 if tag in x.split() else 0)

In [ ]:
data2['image_name']=data2['image_name'].apply(lambda x:'{}.jpg'.format(x))
data2.head()

In [ ]:
data2.columns[2:]

In [ ]:
tgs = list(data2.columns[2:])
tgs

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense,BatchNormalization,Conv2D,MaxPooling2D,Dropout,Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam,SGD
from tensorflow.keras.callbacks import TensorBoard,EarlyStopping,ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
def fscore_beta(y_true,y_pred,beta=2,epsilon=1e-4):
    
    beta_sqr=beta**2
    y_true=tf.cast(y_true,tf.float32)
    y_pred=tf.cast(tf.greater(tf.cast(y_pred,tf.float32),tf.constant(0.5)),tf.float32)
    
    tp=tf.reduce_sum(y_true*y_pred,axis=1)
    fp=tf.reduce_sum(y_pred,axis=1)-tp
    fn=tf.reduce_sum(y_true,axis=1)-tp
    
    pres = tp/(tp+fp+epsilon)
    recall = tp/(tp+fn+epsilon)
    
    fsb = ((1+beta_sqr)*pres*recall)/(beta_sqr*pres+recall+epsilon)
    
    return fsb

In [ ]:
def accur(y_true,y_pred,epsilon=1e-4):
    y_true=tf.cast(y_true,tf.float32)
    y_pred=tf.cast(tf.greater(tf.cast(y_pred,tf.float32),tf.constant(0.5)),tf.float32)
    
    tp=tf.reduce_sum(y_true*y_pred,axis=1)
    fp=tf.reduce_sum(y_pred,axis=1)-tp
    fn=tf.reduce_sum(y_true,axis=1)-tp
    
    y_true=tf.cast(y_true,tf.bool)
    y_pred=tf.cast(y_pred,tf.bool)
    
    tn=tf.reduce_sum(tf.cast(tf.logical_not(y_true),tf.float32)*tf.cast(tf.logical_not(y_pred),tf.float32),axis=1)
    acc = (tp+tn)/(tp+tn+fp+fn+epsilon)
    
    return acc

In [ ]:
def model_build():
    model=Sequential()
    
    model.add(BatchNormalization(input_shape=(128,128,3)))
    model.add(Conv2D(32,kernel_size=(3,3),padding='same',activation='relu'))
    model.add(Conv2D(32,kernel_size=(3,3),activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(64,kernel_size=(3,3),padding='same',activation='relu'))
    model.add(Conv2D(64,kernel_size=(3,3),activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(128,kernel_size=(3,3),padding='same',activation='relu'))
    model.add(Conv2D(128,kernel_size=(3,3),activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(256,kernel_size=(3,3),padding='same',activation='relu'))
    model.add(Conv2D(256,kernel_size=(3,3),activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))
    
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(17, activation='sigmoid'))
    
    opt=Adam(lr=1e-4)
    
    model.compile(loss='binary_crossentropy',
                 optimizer=opt,
                 metrics=[accur,fscore_beta])
    
    return model

In [ ]:
mcp=ModelCheckpoint(filepath = 'best_model.hdf5',
                   monitor = 'val_fbeta',
                   mode = 'max',
                   save_best_only = True,
                   save_weights_only = True)

In [ ]:
image_gen=ImageDataGenerator(rescale=1/255,validation_split=0.2)
train_generator=image_gen.flow_from_dataframe(dataframe=data2,
                                             directory="../input/planets-dataset/planet/planet/train-jpg",
                                             x_col="image_name",
                                             y_col=tgs,
                                             subset="training",
                                             batch_size=16,
                                             seed=2021,
                                             shuffle=True,
                                             class_mode="raw",
                                             target_size=(128,128))

val_generator=image_gen.flow_from_dataframe(dataframe=data2,
                                           directory="../input/planets-dataset/planet/planet/train-jpg",
                                           x_col="image_name",
                                           y_col=tgs,
                                           subset="validation",
                                           batch_size=16,
                                           seed=2021,
                                           shuffle=True,
                                           class_mode="raw",
                                           target_size=(128,128))

In [ ]:
step_train_size=int(np.ceil(train_generator.samples/train_generator.batch_size))
step_val_size=int(np.ceil(val_generator.samples/val_generator.batch_size))

In [ ]:
model1=model_build()
model1.summary()

In [ ]:
model1.fit(x=train_generator,
          steps_per_epoch=step_train_size,
          validation_data=val_generator,
          validation_steps=step_val_size,
          epochs=25,
          callbacks=[mcp])

In [ ]:
! ls ./

In [ ]:
model2=model_build()
model2.load_weights('best_model.hdf5')

In [ ]:
sample_sub=pd.read_csv('../input/planets-dataset/planet/planet/sample_submission.csv')
sample_sub1=sample_sub.copy()
sample_sub1['image_name']=sample_sub1['image_name'].apply(lambda x:'{}.jpg'.format(x))
sample_sub1.head()

In [ ]:
import gc

gc.collect()

In [ ]:
test1_df = sample_sub1.iloc[:40669]['image_name'].reset_index().drop('index', axis =1)
test1_df.head()

In [ ]:
test_image_gen = ImageDataGenerator(rescale = 1/255)

In [ ]:
test_generator1.reset()
pred1 = model2.predict(test_generator1, steps = step_test_size1, verbose = 1)

In [ ]:
file_names1 = test_generator1.filenames

pred_tags1 = pd.DataFrame(pred1)
pred_tags1 = pred_tags1.apply(lambda x: ' '.join(np.array(labels)[x>0.5]), axis = 1)

In [ ]:
result1 = pd.DataFrame({'image_name': file_names1, 'tags': pred_tags1})
result1.head()

In [ ]:
test2_df = sample_submission1.iloc[40669:]['image_name'].reset_index().drop('index', axis =1)
test2_df.head()

In [ ]:
test_generator2 = test_image_gen.flow_from_dataframe(dataframe=test2_df, 
                                                directory="../input/planets-dataset/test-jpg-additional/test-jpg-additional", 
                                                x_col="image_name", 
                                                y_col=None, 
                                                batch_size=16, 
                                                shuffle=False, 
                                                class_mode=None, 
                                                target_size=(128,128))

step_test_size2 = int(np.ceil(test_generator2.samples/test_generator2.batch_size))

In [ ]:
test_generator2.reset()
pred2 = model2.predict(test_generator2, steps = step_test_size2, verbose = 1)

In [ ]:
file_names2 = test_generator2.filenames

pred_tags2 = pd.DataFrame(pred2)
pred_tags2 = pred_tags2.apply(lambda x: ''.join(np.array(labels)[x>0.5]), axis = 1)

In [ ]:
result2 = pd.DataFrame({'image_name': file_names2, 'tags': pred_tags2})
result2.head()

In [ ]:
last_result = pd.concat([result1, result2])

last_result = last_result.reset_index().drop('index', axis =1)

print(last_result.shape)
last_result.head()

In [ ]:
last_result['image_name'] = last_result['image_name'].apply(lambda x: x[:-4])
last_result.head()

In [ ]:
last_result.to_csv('submission_2.csv', index = False)